In [1]:
import functions as fc
import pandas as pd

In [ ]:
fc.download_tgz_raw()

In [ ]:
fc.import_tgz("2018", "./data2018")

In [ ]:
fc.import_tgz("2019", "./data2019")

In [ ]:
fc.import_tgz("2020", "./data2020")

In [ ]:
df_final = fc.transform_rcs("2018")

In [ ]:
df_final.head()

In [ ]:
fc.export_2_minio("2018", df_final)

In [ ]:
fc.export_2_minio("2019", df_final)

In [ ]:
fc.export_2_minio("2020", df_final)

In [3]:
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import requests
from zipfile import ZipFile

def isDirectory(url):
    if(url.endswith('/') and ".." not in url):
        return True
    else:
        return False
list_urls = []
def findLinks(url):
    page = requests.get(url).content
    bsObj = BeautifulSoup(page, 'html.parser')
    maybe_directories = bsObj.findAll('a', href=True)
    list_df = []
    
    for link in maybe_directories:
        #print(link['href'])
        # print(isDirectory(link['href']))
        if(isDirectory(link['href'])):
            newUrl = url + link['href']   
            print(newUrl)   
            list_urls.append(newUrl)  
            print(list_urls) 
            findLinks(newUrl) #recursion happening here$
            
            
        else:
            if(link['href'].endswith('.zip')):
                print("GOTCHA!") #now safe and download
                target_path = link['href']
                response = requests.get(list_urls[-1] + link['href'], stream=True)
                if response.status_code == 200:
                    with open(target_path, 'wb') as f:
                        f.write(response.raw.read())
                
                # zip_file = ZipFile(list_urls[-1] + link['href'])         
                # temp_df = [open_complex_file(zip_file, text_file.filename)
                # for text_file in zip_file.infolist()
                # if text_file.filename.endswith('.csv')]
                # list_df.append(temp_df)

startUrl = "http://data.cquest.org/inpi_rncs/imr/stock/2017/"
findLinks(startUrl)

http://data.cquest.org/inpi_rncs/imr/stock/2017/05/
['http://data.cquest.org/inpi_rncs/imr/stock/2017/05/']
http://data.cquest.org/inpi_rncs/imr/stock/2017/05/04/
['http://data.cquest.org/inpi_rncs/imr/stock/2017/05/', 'http://data.cquest.org/inpi_rncs/imr/stock/2017/05/04/']
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOTCHA!
GOT

In [2]:
import os
path2data=os.getcwd()

In [3]:
path2data

'/home/coder/work/data_inpi/code'

In [4]:
import functions as fc
list_df_rep = fc.import_all_rep(path2data + "/", "*.zip")

/home/coder/work/data_inpi/code/functions.py:44: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep=";")
/home/coder/work/data_inpi/code/functions.py:44: DtypeWarning: Columns (7,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep=";")
/home/coder/work/data_inpi/code/functions.py:44: DtypeWarning: Columns (7,17,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep=";")
/home/coder/work/data_inpi/code/functions.py:44: DtypeWarning: Columns (7,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep=";")
/home/coder/work/data_inpi/code/functions.py:44: DtypeWarning: Columns (7,17,25,32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep="

In [5]:
len(list_df_rep)

134

In [6]:
sys.getsizeof(list_df_rep)

1240

In [7]:
df_final = pd.concat(list_df_rep)

In [8]:
df_final.shape # 9 504 907, 48 variables

(9504907, 48)

In [11]:
import s3fs
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
    
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET_OUT = "radjerad"
FILE_KEY_OUT_S3 = "inpi/" + "data_rep_2017.csv" 
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    df_final.to_csv(file_out, index=False, sep=";", encoding="utf-8")

In [4]:
list_df_pm = fc.import_all_pm(path2data + "/", "*.zip")

/home/coder/work/data_inpi/code/functions.py:44: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep=";")
/home/coder/work/data_inpi/code/functions.py:44: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep=";")
/home/coder/work/data_inpi/code/functions.py:44: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_file.open(fi), sep=";")


In [5]:
len(list_df_pm)

134

In [6]:
df_final_pm = pd.concat(list_df_pm)

In [7]:
df_final_pm.shape # 4 296 552

(4296552, 29)

In [ ]:
import s3fs
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
    
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET_OUT = "radjerad"
FILE_KEY_OUT_S3 = "inpi/" + "data_pm_2017.csv" 
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    df_final_pm.to_csv(file_out, index=False, sep=";", encoding="utf-8")

In [ ]:
def export_2_minio(year, path2output, df_final):
    S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
    BUCKET_OUT = "radjerad"
    FILE_KEY_OUT_S3 = "inpi/" + year + "/" + path2output 
    FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

    with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
        df_final.to_csv(file_out)
    return None


In [ ]:
import pandas as pd
li = []
import s3fs
for i in range(len(list_df)):
    print(i)
    temp = pd.concat(list_df[i])
    print(temp.shape)
    temp = temp.reset_index()
    path2output = temp.loc[0,"source"][0:16]
    print(path2output)
    #temp.to_csv(path2output, sep=";")
    export_2_minio("2017", path2output + ".csv", temp)
    # temp.to_csv(path2output + ".csv", sep=";")
    # li.append(temp)
